<a href="https://colab.research.google.com/github/fasthill/ML-DL-study-alone/blob/main/5-1%20%EA%B2%B0%EC%A0%95%20%ED%8A%B8%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## calculate prediction accuracy

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

In [17]:
import numpy as np
import pandas as pd
import datetime
import joblib
import os, re

In [18]:
import pickle

# write list, dictionary to pickle
def save_to_pickle(path, filename):
    open_file = open(path, "wb")
    pickle.dump(filename, open_file)
    open_file.close()

# read list, dictionary from pickle
def load_from_pickle(path):
    open_file = open(path, "rb")
    loaded_file = pickle.load(open_file)
    open_file.close()
    return loaded_file

In [19]:
# import csv

In [20]:
def is_new_format(f_name): #5월 16일 이전 생성 데이터 확인
    mon = int(f_name[11:13])
    day = int(f_name[13:15])
    if (mon < 5):
        return False
    elif (mon == 5) & (day < 16) :
        return False
    else:
        return True

In [21]:
def find_filename(dir): # find a filename in a directory
    p = re.compile('lgbm_bs_df_(.{9}).*csv')
    dir_list = os.listdir(dir)
    for fname in dir_list:
        aa = p.search(fname)
        if aa is None:
            continue
        else:
            return aa.group()

In [22]:
def find_next_date(current_date):
    current_index = list(OPENING_DAYS_KOR).index(current_date)
    next_date =  OPENING_DAYS_KOR.iloc[current_index+1]
    return next_date

In [33]:
prediction_da = datetime.date(2023, 5, 16) # 예측을 필요로 하는 일자
def calc_result_df(start_id, end_id, current_data, com_data, scaler, model, real_columns):
    predict_value = {}
    p0 = []
    p1 = []
    p2 = []
    p3 = []
    p4 = []
    p5 = []
    p6 = []
    p7 = []
    p8 = []
    p9 = []
    p10 = []
    p11 = []
    p_date = []

    n_count = 0 # 총 갯수
    np_count = 0
    # prediction_date = next_date
    for prediction_date in OPENING_DAYS_KOR[st_id:end_id]:
        prediction_row = current_data[current_data.index == prediction_date]
        if(len(prediction_row) == 0):
#             print("**** No data on", prediction_date, "*****")
            continue
    #     print(prediction_row.iloc[:, 0:5])
        n_count = n_count + 1
        com_row = com_data[com_data['date']  == prediction_date]

        # predict on,off low price, high price
        real_data_df = prediction_row[real_columns] # select necessary columns
        real_scaled = scaler.transform(real_data_df)

        y_predict = model.predict(real_scaled)
        weight = model.predict_proba(real_scaled)

        if y_predict[0] == 1:
            np_count = np_count + 1
    #         print('prediction data: ', prediction_date, 'prediction result :', y_predict[0])
            ratio = com_row['close_cr'].iloc[0]
            if ratio >= 0: # 전일 대비 + 비교
                t_f = True
            else:
                t_f = False

            p0.append(com_name)
            p1.append(y_predict[0])
            p10.append(weight[0,1])
            p11.append(weight[0,0])
#     dict_temp[f'yes'] = f'{weight[0,1]:.2f}'
#     dict_temp[f'no'] = f'{weight[0,0]:.2f}'
            p2.append(t_f)
            p3.append(ratio)
            p_value = com_row['close'].iloc[0] / (1 + ratio/100.)
            p4.append(f'{p_value:.1f}')
            p_c_value = p_value * (1 - 0.005) #  구매 기준: 0.005% 이하로 떨어지면 구매 가능, 주식거래비용 0.003%
            p5.append(f'{p_c_value:.1f}') # 0.5% 적용
            p6.append(com_row['low'].iloc[0])
            p7.append(f"{(p_c_value - com_row['low'].iloc[0]):.1f}")
            if p_c_value >= com_row['low'].iloc[0] :
                p8_yesno = 'yes'
            else:
                p8_yesno = 'no'
            p8.append(p8_yesno)
            if (p8_yesno == 'yes') & t_f :
                final = 'yes'
            else:
                final = 'no'
            p9.append(final)
            p_date.append(prediction_date)

    predict_value['date'] = p_date
    predict_value['company'] = p0
    predict_value['prediction'] = p1
    predict_value['prob_true'] = p10
    predict_value['prob_false'] = p11
    predict_value['result'] = p2
    predict_value['cr_ratio'] = p3
    predict_value['p_close'] = p4
    predict_value['p_c_05%'] = p5
    predict_value['low'] = p6
    predict_value['diff'] = p7
    predict_value['chance_to_buy'] = p8
    predict_value['final'] = p9  # 구매후 이익 취득 가능 판단

#     pre_ratio = np_count / n_count
#     print(f"Count total: {n_count}, count_yes: {np_count}, ratio: {pre_ratio:.2f}")
    result_df = pd.DataFrame(predict_value)
    
    return result_df

In [24]:
import sys, os

module_path = os.path.abspath(os.path.join('.')) # 현재 폴더로 이동
if module_path+"\\data\\base_data\\common_data" not in sys.path:
    sys.path.append(module_path+"\\data\\base_data\\common_data") #  공통으로 사용하는 각종 리스트, 코드 등 
    
import common_data as cd

In [25]:
# get stock market opening days
base_data_directory = './data/base_data/stock_market_holydays/'
OPENING_DAYS_KOR = pd.read_pickle(base_data_directory+'opening_days_kor.pkl') # 한국 개장일 데이터 
OPENING_DAYS_USA = pd.read_pickle(base_data_directory+'opening_days_usa.pkl') # 미국 개장일 데이터 

In [26]:
TRAIN_START_DATE = datetime.date(2022, 3, 2)
TRAIN_END_DATE = datetime.date(2023, 3, 31)

In [27]:
directory_for_predict = './data/data_for_ml/predict/'
directory_for_data = './data/company_pkl/'
directory_for_common = './data/common_pkl/'
directory_for_model = './data/data_for_ml/model/model/'

In [39]:
code_good = {'005930': ['삼성전자', 'sec'], '035420': ['NAVER', 'naver'],
             '005380': ['현대차', 'hyunmotor'], '035720': ['카카오', 'kakao'],
             '000660': ['SK하이닉스', 'skhynix'], '006400': ['삼성SDI', 'sdi'],
             '005935': ['삼성전자우', 'secpre'], '105560': ['KB금융', 'kbbank'],
             '012330': ['현대모비스', 'mobis'],  '055550': ['신한지주', 'shgroup'],
             '003670': ['포스코퓨처엠', 'poscochemical'], '034730': ['SK', 'sk'], 
             '032830': ['삼성생명', 'sslife'], '086790': ['하나금융지주', 'hana'],
             '009150': ['삼성전기', 'sselec'], '017670': ['SK텔레콤', 'sktelecom'],
             '011200': ['HMM', 'hmm'], '000810': ['삼성화재', 'ssfire'], 
             '010950': ['S-Oil', 'soil'], '018260': ['삼성에스디에스', 'sds'],
             '316140': ['우리금융지주', 'woorifg'], '024110': ['기업은행', 'ibk'], 
             '377300': ['카카오페이', 'kakaopay'], '028050': ['삼성엔지니어링', 'ssengineering'],
            }
code_good = {'005930': ['삼성전자', 'sec'], '035420': ['NAVER', 'naver'],
             '005380': ['현대차', 'hyunmotor'], '035720': ['카카오', 'kakao'],
            }
# code_good = {'005930': ['삼성전자', 'sec'], 
#             }

In [29]:
# com_name = 'poscochemical'
TRAIN_START_DATE = datetime.date(2022, 3, 2)
TRAIN_END_DATE = datetime.date(2023, 3, 31)

In [40]:
df_base = pd.DataFrame()

for key, val in code_good.items():
 
    com_name = val[1]
    
    # read selected columns, scaler and model to be appllied

    # locate the model data directory
    directory_model_data = f'{directory_for_model}{com_name}/'

    # get the model data filepath
    columns_pkl = directory_model_data + 'best_columns.pkl' # save with pickle.dump 
    scaler_pkl = directory_model_data + 'best_scaler.pkl'  # one saved with joblib.dump
    scaler_p_pkl = directory_model_data + 'best_scaler_p.pkl' # one saved with pickle.dump
    model_pkl = directory_model_data + 'best_model.pkl'  # one saved with joblib.dump
    model_p_pkl = directory_model_data + 'best_model_p.pkl' # one saved with pickle.dump
    result_pkl = directory_model_data + 'best_result.pkl'

    # load iterating training result data
    result = load_from_pickle(result_pkl) # with pickle.load

    yes_no = is_new_format(find_filename(directory_model_data))
    if yes_no:
        precision = result.loc['test_precision'].iloc[-1]
    else:
        precision = result.loc['precision'].iloc[-1]
        
    # load columns data
    real_columns = load_from_pickle(columns_pkl)[:-5] # column 읽기. target columns 5개는 제외
    # scale the data
    scaler = joblib.load(scaler_pkl) # scaler 읽기
    # apply the scaled real_data to the model
    model = joblib.load(model_pkl) # model 읽기
    
    fname = f'df_{com_name}_combine.pkl'
    f_name = directory_for_predict + fname
    df_o = pd.read_pickle(f_name) 
    com_fname = f'{com_name}_historical.pkl'  # 실제와 예측을 비교하기 위하여 실제데이터을 불러 옴
    f_com_name = directory_for_data + com_fname
    com_data = pd.read_pickle(f_com_name)
    com_data['date'] = com_data['date'].dt.date
    
    current_data = df_o.loc[:, 'retail_1':'weekday'] # select columns except targets columns
    
    next_date = find_next_date(TRAIN_END_DATE) # 최초 시작날짜 설정 (훈련데이터 이후 첫째 날짜)
    
    st_id = list(OPENING_DAYS_KOR).index(next_date) # 훈련데이터 이후 첫째 날짜 index == text data 첫째 날짜
    end_id = list(OPENING_DAYS_KOR).index(df_o.index[-1]) # test data 마지막 날짜
    
    df_com = calc_result_df(st_id, end_id, current_data, com_data, scaler, model, real_columns)
    
    df_base = pd.concat([df_base, df_com],axis=0)

In [41]:
df_base

,date,company,prediction,prob_true,prob_false,result,cr_ratio,p_close,p_c_05%,low,diff,chance_to_buy,final
0,2023-04-12,sec,1,0.537730,0.462270,True,0.15,65901.1,65571.6,65300.0,271.6,yes,yes
1,2023-04-28,sec,1,0.576937,0.423063,True,1.39,64602.0,64279.0,65000.0,-721.0,no,no
2,2023-05-08,sec,1,0.566067,0.433933,True,1.23,65099.3,64773.8,65400.0,-626.2,no,no
3,2023-05-16,sec,1,0.540000,0.460000,True,1.40,64497.0,64174.6,65300.0,-1125.4,no,no
4,2023-05-18,sec,1,0.562701,0.437299,True,1.85,64997.5,64672.6,65700.0,-1027.4,no,no
5,2023-06-14,sec,1,0.519216,0.480784,False,-0.14,72000.8,71640.8,71100.0,540.8,yes,no
0,2023-04-14,naver,1,0.506751,0.493249,True,1.32,196407.4,195425.4,197000.0,-1574.6,no,no
1,2023-04-28,naver,1,0.567817,0.432183,True,2.18,188197.3,187256.3,190000.0,-2743.7,no,no
2,2023-05-09,naver,1,0.567817,0.432183,True,1.93,207004.8,205969.8,208500.0,-2530.2,no,no
3,2023-05-11,naver,1,0.506751,0.493249,True,0.48,209992.0,208942.1,210500.0,-1557.9,no,no


In [34]:
df_base = pd.DataFrame()
df_base_nega = pd.DataFrame()

for key, val in code_good.items():
 
    com_name = val[1]
    
    # read selected columns, scaler and model to be appllied

    # locate the model data directory
    directory_model_data = f'{directory_for_model}{com_name}/'

    # get the model data filepath
    columns_pkl = directory_model_data + 'best_columns.pkl' # save with pickle.dump 
    scaler_pkl = directory_model_data + 'best_scaler.pkl'  # one saved with joblib.dump
    scaler_p_pkl = directory_model_data + 'best_scaler_p.pkl' # one saved with pickle.dump
    model_pkl = directory_model_data + 'best_model.pkl'  # one saved with joblib.dump
    model_p_pkl = directory_model_data + 'best_model_p.pkl' # one saved with pickle.dump
    result_pkl = directory_model_data + 'best_result.pkl'
    
    # get the negative model data filepath
    columns_nega_pkl = directory_model_data + 'best_columns_nega.pkl' # save with pickle.dump 
    scaler_nega_pkl = directory_model_data + 'best_scaler_nega.pkl'  # one saved with joblib.dump
    scaler_p_nega_pkl = directory_model_data + 'best_scaler_p_nega.pkl' # one saved with pickle.dump
    model_nega_pkl = directory_model_data + 'best_model_nega.pkl'  # one saved with joblib.dump
    model_p_nega_pkl = directory_model_data + 'best_model_p_nega.pkl' # one saved with pickle.dump
    result_nega_pkl = directory_model_data + 'best_result_nega.pkl'

    # load iterating training result data
    result = load_from_pickle(result_pkl) # with pickle.load
    result_nega = load_from_pickle(result_nega_pkl) # with pickle.load

    yes_no = is_new_format(find_filename(directory_model_data))
    if yes_no:
        precision = result.loc['test_precision'].iloc[-1]
        precision_nega = result_nega.loc['test_precision'].iloc[-1]
    else:
        precision = result.loc['precision'].iloc[-1]
        precision_nega = result_nega.loc['precision'].iloc[-1]
        
    # load columns data
    real_columns = load_from_pickle(columns_pkl)[:-5] # column 읽기. target columns 5개는 제외
    real_columns_nega = load_from_pickle(columns_nega_pkl)[:-5] # column 읽기. target columns 5개는 제외
    # scale the data
    scaler = joblib.load(scaler_pkl) # scaler 읽기
    scaler_nega = joblib.load(scaler_nega_pkl) # scaler 읽기
    # apply the scaled real_data to the model
    model = joblib.load(model_pkl) # model 읽기
    model_nega = joblib.load(model_nega_pkl) # model 읽기
    
    fname = f'df_{com_name}_combine.pkl'
    f_name = directory_for_predict + fname
    df_o = pd.read_pickle(f_name) 
    com_fname = f'{com_name}_historical.pkl'  # 실제와 예측을 비교하기 위하여 실제데이터을 불러 옴
    f_com_name = directory_for_data + com_fname
    com_data = pd.read_pickle(f_com_name)
    com_data['date'] = com_data['date'].dt.date
    
    current_data = df_o.loc[:, 'retail_1':'weekday'] # select columns except targets columns
    
    next_date = find_next_date(TRAIN_END_DATE) # 최초 시작날짜 설정 (훈련데이터 이후 첫째 날짜)
    
    st_id = list(OPENING_DAYS_KOR).index(next_date) # 훈련데이터 이후 첫째 날짜 index == text data 첫째 날짜
    end_id = list(OPENING_DAYS_KOR).index(df_o.index[-1]) # test data 마지막 날짜
    
    df_com = calc_result_df(st_id, end_id, current_data, com_data, scaler, model, real_columns)
    df_com_nega = calc_result_df(st_id, end_id, current_data, com_data, scaler_nega, model_nega, real_columns_nega)
    
    df_base = pd.concat([df_base, df_com],axis=0)
    df_base_nega = pd.concat([df_base_nega, df_com_nega],axis=0)



In [35]:
df_base

,date,company,prediction,prob_true,prob_false,result,cr_ratio,p_close,p_c_05%,low,diff,chance_to_buy,final
0,2023-04-12,sec,1,0.537730,0.462270,True,0.15,65901.1,65571.6,65300.0,271.6,yes,yes
1,2023-04-28,sec,1,0.576937,0.423063,True,1.39,64602.0,64279.0,65000.0,-721.0,no,no
2,2023-05-08,sec,1,0.566067,0.433933,True,1.23,65099.3,64773.8,65400.0,-626.2,no,no
3,2023-05-16,sec,1,0.540000,0.460000,True,1.40,64497.0,64174.6,65300.0,-1125.4,no,no
4,2023-05-18,sec,1,0.562701,0.437299,True,1.85,64997.5,64672.6,65700.0,-1027.4,no,no
5,2023-06-14,sec,1,0.519216,0.480784,False,-0.14,72000.8,71640.8,71100.0,540.8,yes,no


In [36]:
df_base_nega

,date,company,prediction,prob_true,prob_false,result,cr_ratio,p_close,p_c_05%,low,diff,chance_to_buy,final
0,2023-04-05,sec,1,0.566224,0.433776,True,0.47,63601.1,63283.1,63400.0,-116.9,no,no
1,2023-04-06,sec,1,0.526165,0.473835,False,-2.50,63897.4,63577.9,62300.0,1277.9,yes,no
2,2023-04-13,sec,1,0.525530,0.474470,True,0.15,66001.0,65671.0,65400.0,271.0,yes,yes
3,2023-04-14,sec,1,0.504225,0.495775,False,-1.51,66098.1,65767.6,65000.0,767.6,yes,no
4,2023-04-17,sec,1,0.557633,0.442367,True,0.31,65098.2,64772.7,64700.0,72.7,yes,yes
5,2023-04-20,sec,1,0.522585,0.477415,False,-0.31,65503.1,65175.5,64600.0,575.5,yes,no
6,2023-04-21,sec,1,0.553406,0.446594,True,0.61,65301.7,64975.2,65400.0,-424.8,no,no
7,2023-04-24,sec,1,0.521013,0.478987,False,-0.76,65699.3,65370.8,64800.0,570.8,yes,no
8,2023-04-25,sec,1,0.516488,0.483512,False,-2.45,65197.3,64871.3,63400.0,1471.3,yes,no
9,2023-04-26,sec,1,0.552014,0.447986,True,0.79,63597.6,63279.6,63300.0,-20.4,no,no


In [16]:
df_base['result'].value_counts()

True     22
False     4
Name: result, dtype: int64

In [17]:
df_benifit = df_base[df_base['final'] == 'yes']

In [18]:
df_benifit.sort_values(by=['date']).reset_index(drop=True) # 구매후 이익실현 경우

,date,company,prediction,result,cr_ratio,p_close,p_c_05%,low,diff,chance_to_buy,final
0,2023-04-12,sec,1,True,0.15,65901.1,65571.6,65300.0,271.6,yes,yes
1,2023-04-19,hyunmotor,1,True,1.15,191398.9,190441.9,190300.0,141.9,yes,yes
2,2023-06-02,naver,1,True,0.25,203990.0,202970.1,202500.0,470.1,yes,yes
3,2023-06-08,hyunmotor,1,True,0.15,196405.4,195423.4,194000.0,1423.4,yes,yes


In [19]:
df_lost = df_base[(~df_base['result']) & (df_base['chance_to_buy'] == 'yes')]

In [20]:
df_lost.sort_values(by=['date']).reset_index(drop=True)  # 구매후 실퍠 경우

,date,company,prediction,result,cr_ratio,p_close,p_c_05%,low,diff,chance_to_buy,final
0,2023-05-16,hyunmotor,1,False,-2.64,207991.0,206951.0,201000.0,5951.0,yes,no
1,2023-05-18,naver,1,False,-0.23,212989.9,211924.9,210000.0,1924.9,yes,no
2,2023-05-30,naver,1,False,-0.49,202492.2,201479.8,200500.0,979.8,yes,no
3,2023-06-14,sec,1,False,-0.14,72000.8,71640.8,71100.0,540.8,yes,no


In [ ]:
code = cd.code_all # 전체 회사 코드

code_mid = {'373220': ['LG에너지솔루션', 'lgenergy'], '207940': ['삼성바이오로직스', 'ssbio'],
            '000270': ['기아', 'kia'], '028260': ['삼성물산', 'sscnt'],
            '015760': ['한국전력', 'koreaelec'], '034020': ['두산에너빌리티', 'doosanener'],
            '051900': ['LG생활건강', 'lglife'], '259960': ['크래프톤', 'crafton'],
            '361610': ['SK아이이테크놀로지', 'skietech'], '086280': ['현대글로비스', 'glovis'],
            '302440': ['SK바이오사이언스', 'skbio'],
            }

code_bad = {'051910': ['LG화학', 'lgchemical'], '033780': ['KT&G', 'ktng'],
            '005490': ['POSCO홀딩스', 'poscoholding'], '068270': ['셀트리온', 'celltrion'],
            '066570': ['LG전자', 'lgelec'],  '096770': ['SK이노베이션', 'skinnovation'],
            '030200': ['KT', 'kt'], '003550': ['LG', 'lg'],
            '329180': ['현대중공업', 'hhi'], '003490': ['대한항공', 'koreanair'],
            '036570': ['엔씨소프트', 'ncsoft'], '009830': ['한화솔루션', 'hanhwasol'],
            '090430': ['아모레퍼시픽', 'amore'], '011170': ['롯데케미칼', 'lottechem'],
            '138040': ['메리츠금융지주', 'meritz'], '011070': ['LG이노텍', 'lginnotek'],
           }

code_good = {'005930': ['삼성전자', 'sec'], '035420': ['NAVER', 'naver'],
             '005380': ['현대차', 'hyunmotor'], '035720': ['카카오', 'kakao'],
             '000660': ['SK하이닉스', 'skhynix'], '006400': ['삼성SDI', 'sdi'],
             '005935': ['삼성전자우', 'secpre'], '105560': ['KB금융', 'kbbank'],
             '012330': ['현대모비스', 'mobis'],  '055550': ['신한지주', 'shgroup'],
             '003670': ['포스코퓨처엠', 'poscochemical'], '034730': ['SK', 'sk'], 
             '032830': ['삼성생명', 'sslife'], '086790': ['하나금융지주', 'hana'],
             '009150': ['삼성전기', 'sselec'], '017670': ['SK텔레콤', 'sktelecom'],
             '011200': ['HMM', 'hmm'], '000810': ['삼성화재', 'ssfire'], 
             '010950': ['S-Oil', 'soil'], '018260': ['삼성에스디에스', 'sds'],
             '316140': ['우리금융지주', 'woorifg'], '024110': ['기업은행', 'ibk'], 
             '377300': ['카카오페이', 'kakaopay'], '028050': ['삼성엔지니어링', 'ssengineering'],
            }